In [18]:
import sqlite3
import pandas as pd

def print_table(query, p=False):
    conn = sqlite3.connect('../data/ratingRager.db')
    cursor = conn.cursor()
    cursor.execute(query)

    column_names = [description[0] for description in cursor.description]

    rows = cursor.fetchall()

    column_widths = [max(len(str(row[i])) for row in rows + [column_names]) for i in range(len(column_names))]

    header = "  ".join(f"{name:<{column_widths[i]}}" for i, name in enumerate(column_names))
    if p:
        print('\n',header)
        print("-" * len(header))

        for row in rows:
            row_str = "  ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row))
            print(row_str)
        print('\n')

    return pd.read_sql_query(query, conn)

In [9]:
QUERY = """
    SELECT * FROM scores;
"""
df = print_table(QUERY)

In [10]:
# Overall Score = 0.2 * sentiment_score + 0.25 * trend_score + 0.1 * location_score + 0.35 * rating_score + 0.1 * price_score

df["overall_score"] = (0.25 * df["sentiment_score"]) + (0.05 * df["trend_score"]) + (0.1 * df["location_score"]) + (0.4 * df["rating_score"]) + (0.05 * df["price_score"]) + (0.15 * df["uniqueness_score"])

In [ ]:
# Update score table

conn = sqlite3.connect('../data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET overall_score = ? WHERE restaurant_id = ?;",
        (row['overall_score'], row['restaurant_id'])
    )

conn.commit()
conn.close()

In [12]:
QUERY = """
    SELECT r.id, r.name, s.overall_score FROM restaurants as r
    JOIN scores as s ON r.id=s.restaurant_id
    ORDER BY s.overall_score DESC
"""
df = print_table(QUERY)


In [ ]:
conn = sqlite3.connect('../data/ratingRager.db')
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute("UPDATE restaurants SET ranking = ? WHERE id = ?", (index+1, row['id']))

conn.commit()
conn.close()

In [20]:
QUERY = """
    SELECT r.id, r.name, s.overall_score, r.ranking, r.latitude, r.longitude FROM restaurants as r
    JOIN scores as s ON r.id=s.restaurant_id
    WHERE overall_score > 0.8
    ORDER BY s.overall_score DESC;
"""
df = print_table(QUERY, p=True)


TypeError: 'bool' object is not callable

In [16]:
import folium

m = folium.Map(location=[51.0447, -114.0719], zoom_start=12)  # Coords for Calgary

# Add restaurants as points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(f"Name: {row['name']}<br>Ranking: {row['ranking']} <br>Overall Score: {row['overall_score']:.2f}", max_width=200),
    ).add_to(m)

# Display the map
m.save('map.html')
m
